# Damatura, Nigeria IDP Analysis
##### January 2025

In [1]:
# standard library imports
import os
import sys

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# third-party imports
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd

# change to project root directory
os.chdir('../')
print("directory changes")
print(f"current working directory is:", os.getcwd())

# local imports (after changing directory)
from src.population import (
    clip_raster_with_shapefile,
    raster_to_points,
    raster_to_polygons,
    pd_calc_raster_to_polygons
)

from src.idp import kmz_to_shapefile, transform_idp_description, transform_idp_complete

directory changes
current working directory is: /Users/carolinecullinan/dev/wb/damaturu-nigeria-idp


# 1.0 Population Density

## 1.1 Population Density Data (1 km resolution)

In [2]:
# check coordinate system of population density data for Nigeria, nga_pd__2020_1km_UNadj.tif
with rasterio.open("data/raw/nga_pd_2020_1km_UNadj.tif") as src:
    print(f"coordinate system: {src.crs}")

coordinate system: EPSG:4326


In [3]:
# check coordinate system of aoi, damaturu.shp
aoi = gpd.read_file("data/raw/AOI/damaturu.shp")
print(f"coordinate system: {aoi.crs}")

coordinate system: EPSG:4326


### 1.1.1 Raster to Points Conversion

In [2]:
# apply "clip_raster_with_shapefile" function to get population density (raster) 
## of Damaturu, Nigeria specifically (shapefile) and sav interim "damaturu_pd" (raster) tif format
clip_raster_with_shapefile(
    "data/raw/nga_pd_2020_1km_UNadj.tif",
    "data/raw/AOI/damaturu.shp",
    "data/interim/damaturu_pd.tif"
)

In [3]:
# gain insight into raster dimensions to best choose a sample rate for raster to point conversion
## (i.e., determine how many points to sample from the raster, 
## where sample_rate = 1 means that every pixel from the raster is converted to a point)
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    print(f"raster dimensions: {src.width} x {src.height}")
    print(f"total pixels: {src.width * src.height}")
    
# note: raster dimensions and total pixels are small, therefore use sample_rate = 1

raster dimensions: 17 x 13
total pixels: 221


In [8]:
with rasterio.open("data/interim/damaturu_pd.tif") as src:
    total_pixels = src.width * src.height
    target_points = 20000  # adjust based on your needs (web viz, aim for 10,000-50,000 points)
    suggested_sample_rate = int(total_pixels / target_points)
    print(f"suggested sample rate: {suggested_sample_rate}")

suggested sample rate: 0


In [4]:
# convert interim (raster) "damaturu_pd" to processed "damaturu_pd_points" points for export and viz
raster_to_points(
    "data/interim/damaturu_pd.tif",
    'data/processed/damaturu_pd_points.geojson',
    sample_rate=1
)

input raster dimensions: 17 x 13
total pixels: 221

Created 221 points
Value range: -99999.00 to 3183.82


,population_density,geometry
0,-99999.0,POINT (11.91125 11.79542)
1,-99999.0,POINT (11.91958 11.79542)
2,-99999.0,POINT (11.92792 11.79542)
3,-99999.0,POINT (11.93625 11.79542)
4,-99999.0,POINT (11.94458 11.79542)
...,...,...
216,-99999.0,POINT (12.01125 11.69542)
217,-99999.0,POINT (12.01958 11.69542)
218,-99999.0,POINT (12.02792 11.69542)
219,-99999.0,POINT (12.03625 11.69542)


### 1.1.2 Raster to Polygon Conversion

In [4]:
# convert Damaturu specific population density raster to polygon GeoJSON for export and viz
raster_to_polygons(
    "data/raw/nga_pd_2020_1km_UNadj.tif",
    "data/raw/AOI/damaturu.shp",
    "data/processed/damaturu_poly.geojson"
)


Created 95 polygons
Value range: 89.82 to 3183.82


,geometry,population_density
0,"POLYGON ((11.97375 11.79125, 11.97375 11.78292...",132.874374
1,"POLYGON ((11.98208 11.79125, 11.98208 11.78292...",118.977905
2,"POLYGON ((11.96542 11.78292, 11.96542 11.77458...",249.598358
3,"POLYGON ((11.97375 11.78292, 11.97375 11.77458...",171.849319
4,"POLYGON ((11.98208 11.78292, 11.98208 11.77458...",165.467773
...,...,...
91,"POLYGON ((11.92375 11.70792, 11.92375 11.69958...",91.995384
92,"POLYGON ((11.93208 11.70792, 11.93208 11.69958...",142.913956
93,"POLYGON ((11.94042 11.70792, 11.94042 11.69958...",224.143234
94,"POLYGON ((11.94875 11.70792, 11.94875 11.69958...",401.490784


## 1.2 Population Data (100 m resolution)

In [2]:
#calc
pd_calc_raster_to_polygons(
    "data/raw/nga_ppp_2020_UNadj.tif",  # 100m population raster
    "data/raw/AOI/damaturu.shp",     #  AOI shapefile
    "data/processed/damaturu_100m_poly.geojson"  # output path
)

Values in clipped area before processing:
Unprocessed min in area: -99999.0
Unprocessed max in area: 1016.6382446289062
Density raster min: 0.4713766276836395
Density raster max: 99999.0

Created 9896 polygons
Value range: 0.47 to 99999.00


,geometry,population_density
0,"POLYGON ((11.98625 11.79292, 11.98625 11.79208...",0.763216
1,"POLYGON ((11.98708 11.79292, 11.98708 11.79208...",0.755720
2,"POLYGON ((11.98792 11.79292, 11.98792 11.79208...",0.744945
3,"POLYGON ((11.98875 11.79292, 11.98875 11.79208...",0.741613
4,"POLYGON ((11.98958 11.79292, 11.98958 11.79208...",0.722886
...,...,...
9891,"POLYGON ((11.94625 11.69958, 11.94625 11.69875...",1.546591
9892,"POLYGON ((11.94708 11.69958, 11.94708 11.69875...",1.541078
9893,"POLYGON ((11.94792 11.69958, 11.94792 11.69875...",1.591542
9894,"POLYGON ((11.94875 11.69958, 11.94875 11.69875...",1.583845


In [3]:
with rasterio.open("data/raw/nga_ppp_2020_UNadj.tif") as src:
    data = src.read(1)  # read first band
    print(f"Original raster min: {data.min()}")
    print(f"Original raster max: {data.max()}")

Original raster min: -99999.0
Original raster max: 1016.6382446289062


In [4]:
damaturu_pd100m_poly = gpd.read_file("data/processed/damaturu_pd100m_poly.geojson")
print(f"Min population: {damaturu_pd100m_poly['population_density'].min()}")
print(f"Max population: {damaturu_pd100m_poly['population_density'].max()}")

Min population: 0.4713766276836395
Max population: 32.23027801513672


# 2.0 IDP (Location, Location by Site Type, and Change in IDP Locations by Site Types (i.e, "IDP Growth"))

## 2.1 KMZ to Shapefile Conversion

In [4]:
# convert IDP location KMZ to shapefile
idp_shapefile = kmz_to_shapefile(
    "data/raw/Round_39_North_East_IDP_Camp_Sites_KMZ.kmz",
    "data/interim/idp_shapefile.shp"
)

/Users/carolinecullinan/dev/wb/damaturu-nigeria-idp/src/idp.py:31: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shp)


## 2.2 IDP Damaturu Clip

In [3]:
# check coordinate system of IDP shapefile
print(f"coordinate system: {idp_shapefile.crs}")

coordinate system: EPSG:4326


In [5]:
# clip IDP shapefile to Damaturu shapefile
damaturu_idp = gpd.clip(idp_shapefile, aoi)

In [9]:
# save damaturu_idp to shapefile
damaturu_idp.to_file("data/interim/damaturu_idp.shp")

/var/folders/xy/9wgvl7914cj_8zq0xmgxg4hw0000gn/T/ipykernel_86795/3013792111.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  damaturu_idp.to_file("data/processed/damaturu_idp.shp")


### 2.3 IDP Damaturu Exploration

In [3]:
# map and explore damaturu_idp
damaturu_idp.explore()

NameError: name 'damaturu_idp' is not defined

In [7]:
# gain insight into damaturu_idp
print(damaturu_idp.head())
print(damaturu_idp.columns)
print(damaturu_idp[['Description']])


                  Name                                        Description  \
293   MOH'D GOMBE FARM  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
289   BUKAR ABBA HOUSE  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
288        ABARI Y.B.C  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   
292  HURI VILLAGE CAMP  <html xmlns:fo="http://www.w3.org/1999/XSL/For...   

                                geometry  
293  POINT Z (12.03584 11.72675 0.00000)  
289  POINT Z (11.98183 11.72985 0.00000)  
288  POINT Z (11.98050 11.72993 0.00000)  
292  POINT Z (12.01215 11.73905 0.00000)  
Index(['Name', 'Description', 'geometry'], dtype='object')
                                           Description
293  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
289  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
288  <html xmlns:fo="http://www.w3.org/1999/XSL/For...
292  <html xmlns:fo="http://www.w3.org/1999/XSL/For...


In [20]:
# damaturu_idp shapefile to dataframe for easy data analysis
damaturu_idp_df = transform_idp_description(damaturu_idp)

In [ ]:
# save damaturu_idp_df to csv
damaturu_idp_df.to_csv("data/interim/damaturu_idp_df.csv", index=False)

In [15]:
# damaturu_idp shapefile to clean geojson with all attributes as columns for easy mapping
damaturu_idp_clean = transform_idp_complete(damaturu_idp)



In [16]:
# gain insight into damaturu_idp_clean and look at what columns we have
print("Columns in transformed data:")
print(damaturu_idp_clean.columns)

# gain insight into damaturu_idp_clean and look at first row
print("\nFirst site data:")
print(damaturu_idp_clean.iloc[0])

Columns in transformed data:
Index(['name', 'dtm_round', 'date', 'state', 'state_pcode', 'lga', 'ward',
       'siteid', 'site_name', 'longitude',
       ...
       'percent_of_children_attending_school',
       'occupation_of_majority_of_idps', '#_of_hh_with_source_of_income',
       'access_to_income_generating_activities', 'livestock_on_site',
       'idps_have_access_to_land_for_cultivation',
       'most_trusted_source_of_info',
       'main_topic_community_is_requesting_info',
       'serious_problem_due_to_lack_information', 'geometry'],
      dtype='object', length=102)

First site data:
name                                                     MOH'D GOMBE FARM
dtm_round                                                             R39
date                                                           06/10/2021
state                                                                YOBE
state_pcode                                                         NG036
                           

In [21]:
# gain insight into damaturu_idp_clean 
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_clean)

,name,dtm_round,date,state,state_pcode,lga,ward,siteid,site_name,longitude,latitude,site_status,site_type,site_classification,approximate_site_area,site_management_agency,sma_type,registration_activity,wash_support,cccm_support,cccm_organization,health_support,shelter_support,nfi_support,food_support,protection_support,education_support,livelihood_support,pss_support,pss_organization,land_ownership,most_common_type_of_shelter,number_of_households,number_of_individuals,boys_<1,girls_<1,boys_1___5_y,girls_1___5_y,boys_6to12y,girls_6to12y,boys_13to17y,girls_13to17y,men_18___59_y,women_18___59_y,elderly_men_60+_y,elderly_women_60+_y,have_idps_been_displaced_previously,reason_for_displacement_of_majority,natural_hazard_risks,intended_area_of_return,reasons_to_return_home,reasons_not_to_return_home,percent_hh_living_outside_no_shelter,percent_hh_living_in_ems,percent_hh_living_in_makeshift_self_made_shelter,percent_hh_living_indoors_solid_walls,percent_hh_have_access_to_electricity,percent_hh_have_access_to_safe_cooking_facilities,percent_hh_have_private_living_area,percent_hh_have_mosquito_nets,most_needed_nfi,most_unmet_needs,location_of_site_main_water_source,main_drinking_water_source,avg_amount_of_water_available__day_person,is_drinking_water_potable,complaints_about_drinking_water_quality,condition_of_most_of_the_toilets,#_of_functioning_toilets,main_garbage_disposal,garbage_and_solid_waste_problem,availability_of_hand_washing_station,hygiene_promotion_campaign,evidence_of_open_defecation,access_to_food,access_to_market_near_from_the_site,frequency_of_food_distribution,most_common_source_for_obtaining_food,screening_for_malnutrition,most_health_problem,regular_access_to_medicine,access_to_health_facility,location_of_health_facilities,main_provider_of_health_facilities,main_provider_of_health_services,primary_health_care_for_children,primary_health_care_for_adult,secondary_health_care_for_children,secondary_health_care_for_adult,traditional_health_care,access_to_education_services,location_of_education_facilities,percent_of_children_attending_school,occupation_of_majority_of_idps,#_of_hh_with_source_of_income,access_to_income_generating_activities,livestock_on_site,idps_have_access_to_land_for_cultivation,most_trusted_source_of_info,main_topic_community_is_requesting_info,serious_problem_due_to_lack_information,geometry
0,MOH'D GOMBE FARM,R39,06/10/2021,YOBE,NG036,DAMATURU,MAISANDARI/WAZIRI IBRAHIM ESTATE,YB_S004,MOH'D GOMBE FARM,12.03584,11.72675,informal,Collective Settlement/Centre,Spontaneous,"183,444 sq.m",False,none,True,True,False,No CCCM Organization,False,True,False,True,False,True,False,False,No PSS ORGANIZATION,private building,individual house,55.0,330.0,8.0,10.0,11.0,15.0,16.0,20.0,39.0,46.0,63.0,88.0,6.0,8.0,False,insurgency,none,place of origin,improved security,I want to go back,none,none,none,>75%,none,25%_50%,none,51%_75%,kitchen sets,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),25.0,garbage pit,False,none,False,False,yes onsite,True,once_a_month,cultivated,False,malaria,False,False,off-site (> 3 km),government,government,False,True,False,False,False,True,onsite,<25%,agro-pastoralism,0.0,False,True,True,government official,safety and security,True,POINT Z (12.03584 11.72675 0.00000)
1,BUKAR ABBA HOUSE,R39,08/10/2021,YOBE,NG036,DAMATURU,DAMATURU CENTRAL,YB_S021,BUKAR ABBA HOUSE,11.98183,11.72985,informal,Collective Settlement/Centre,Spontaneous,"65,276 sq.m",False,none,True,True,False,No CCCM Organization,True,True,True,False,False,True,True,False,No PSS ORGANIZATION,Public/Government,individual house,78.0,468.0,19.0,25.0,25.0,27.0,28.0,31.0,23.0,27.0,104.0,149.0,4.0,6.0,False,insurgency,none,none stay here,improved security,I want to go back,none,none,none,>75%,25%_50%,51%_75%,<25%,51%_75%,mattress,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),5.0,garbage pit,False,none,False,False,False,False,never,cash (personal mone

In [14]:
# save damaturu_idp_clean to geojson for mapping
damaturu_idp_clean.to_file('data/processed/damaturu_idp_clean.geojson', driver='GeoJSON')

In [3]:
# read in damaturu_idp_clean and exlore
damaturu_idp_clean = gpd.read_file("data/processed/damaturu_idp_clean.geojson")

# map and explore damaturu_idp_clean
damaturu_idp_clean.explore()

In [4]:
# gain insight into damaturu_idp_clean 
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(damaturu_idp_clean)

,name,dtm_round,date,state,state_pcode,lga,ward,siteid,site_name,longitude,latitude,site_status,site_type,site_classification,approximate_site_area,site_management_agency,sma_type,registration_activity,wash_support,cccm_support,cccm_organization,health_support,shelter_support,nfi_support,food_support,protection_support,education_support,livelihood_support,pss_support,pss_organization,land_ownership,most_common_type_of_shelter,number_of_households,number_of_individuals,boys_<1,girls_<1,boys_1___5_y,girls_1___5_y,boys_6to12y,girls_6to12y,boys_13to17y,girls_13to17y,men_18___59_y,women_18___59_y,elderly_men_60+_y,elderly_women_60+_y,have_idps_been_displaced_previously,reason_for_displacement_of_majority,natural_hazard_risks,intended_area_of_return,reasons_to_return_home,reasons_not_to_return_home,percent_hh_living_outside_no_shelter,percent_hh_living_in_ems,percent_hh_living_in_makeshift_self_made_shelter,percent_hh_living_indoors_solid_walls,percent_hh_have_access_to_electricity,percent_hh_have_access_to_safe_cooking_facilities,percent_hh_have_private_living_area,percent_hh_have_mosquito_nets,most_needed_nfi,most_unmet_needs,location_of_site_main_water_source,main_drinking_water_source,avg_amount_of_water_available__day_person,is_drinking_water_potable,complaints_about_drinking_water_quality,condition_of_most_of_the_toilets,#_of_functioning_toilets,main_garbage_disposal,garbage_and_solid_waste_problem,availability_of_hand_washing_station,hygiene_promotion_campaign,evidence_of_open_defecation,access_to_food,access_to_market_near_from_the_site,frequency_of_food_distribution,most_common_source_for_obtaining_food,screening_for_malnutrition,most_health_problem,regular_access_to_medicine,access_to_health_facility,location_of_health_facilities,main_provider_of_health_facilities,main_provider_of_health_services,primary_health_care_for_children,primary_health_care_for_adult,secondary_health_care_for_children,secondary_health_care_for_adult,traditional_health_care,access_to_education_services,location_of_education_facilities,percent_of_children_attending_school,occupation_of_majority_of_idps,#_of_hh_with_source_of_income,access_to_income_generating_activities,livestock_on_site,idps_have_access_to_land_for_cultivation,most_trusted_source_of_info,main_topic_community_is_requesting_info,serious_problem_due_to_lack_information,geometry
0,MOH'D GOMBE FARM,R39,06/10/2021,YOBE,NG036,DAMATURU,MAISANDARI/WAZIRI IBRAHIM ESTATE,YB_S004,MOH'D GOMBE FARM,12.03584,11.72675,informal,Collective Settlement/Centre,Spontaneous,"183,444 sq.m",False,none,True,True,False,No CCCM Organization,False,True,False,True,False,True,False,False,No PSS ORGANIZATION,private building,individual house,55.0,330.0,8.0,10.0,11.0,15.0,16.0,20.0,39.0,46.0,63.0,88.0,6.0,8.0,False,insurgency,none,place of origin,improved security,I want to go back,none,none,none,>75%,none,25%_50%,none,51%_75%,kitchen sets,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),25.0,garbage pit,False,none,False,False,yes onsite,True,once_a_month,cultivated,False,malaria,False,False,off-site (> 3 km),government,government,False,True,False,False,False,True,onsite,<25%,agro-pastoralism,0.0,False,True,True,government official,safety and security,True,POINT Z (12.03584 11.72675 0.00000)
1,BUKAR ABBA HOUSE,R39,08/10/2021,YOBE,NG036,DAMATURU,DAMATURU CENTRAL,YB_S021,BUKAR ABBA HOUSE,11.98183,11.72985,informal,Collective Settlement/Centre,Spontaneous,"65,276 sq.m",False,none,True,True,False,No CCCM Organization,True,True,True,False,False,True,True,False,No PSS ORGANIZATION,Public/Government,individual house,78.0,468.0,19.0,25.0,25.0,27.0,28.0,31.0,23.0,27.0,104.0,149.0,4.0,6.0,False,insurgency,none,none stay here,improved security,I want to go back,none,none,none,>75%,25%_50%,51%_75%,<25%,51%_75%,mattress,Food,on-site (<10 minutes),piped water supply,unknown,True,False,not so good (not hygienic),5.0,garbage pit,False,none,False,False,0,False,never,cash (personal money),F